In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.models import load_model
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
import seaborn as sns

In [2]:
int2emotions = {0:'angry',1:'fear',2:'happy',3:'heutral',4:'sad',5:'surprise',6:'disgust'}
emotions2int = {'angry':0,'fear':1,'happy':2,'neutral':3,'sad':4,'surprise':5,'disgust':6}

dic = {'images':[], 'labels':[], 'purpose':[]}
    
for d in os.listdir('C:/Users/karth/OneDrive/Desktop/emotion 3/'):
    for emotion in os.listdir(f'C:/Users/karth/OneDrive/Desktop/emotion 3/fer2013/{d}'):
        print(emotion)
        for i in os.listdir(f'C:/Users/karth/OneDrive/Desktop/emotion 3/fer2013/{d}/{emotion}'):
            img = cv2.imread(f'C:/Users/karth/OneDrive/Desktop/emotion 3/fer2013/{d}/{emotion}/{i}',0)
            img = img.reshape(48,48,1)
            dic['images'].append(img)
            dic['labels'].append(emotion)
            
            if d=='train':
                dic['purpose'].append('T')
            else:
                dic['purpose'].append('V')

df = pd.DataFrame(dic)
df.head()

angry
disgust
fear
happy
neutral
sad
surprise
angry
disgust
fear
happy
neutral
sad
surprise


,images,labels,purpose
0,"[[[46], [34], [26], [23], [12], [32], [35], [2...",angry,V
1,"[[[55], [33], [54], [32], [26], [21], [50], [2...",angry,V
2,"[[[123], [123], [126], [131], [124], [69], [10...",angry,V
3,"[[[255], [255], [255], [255], [255], [255], [2...",angry,V
4,"[[[127], [121], [124], [137], [123], [118], [1...",angry,V


In [3]:
train_data = df[df['purpose']=='T']
val_data = df[df['purpose']=='V']

In [4]:
train_data.head()

,images,labels,purpose
7178,"[[[50], [32], [15], [11], [12], [10], [10], [8...",angry,T
7179,"[[[29], [24], [29], [11], [15], [22], [26], [1...",angry,T
7180,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",angry,T
7181,"[[[155], [138], [98], [112], [94], [97], [111]...",angry,T
7182,"[[[211], [215], [220], [221], [219], [216], [2...",angry,T


In [5]:
val_data.head()

,images,labels,purpose
0,"[[[46], [34], [26], [23], [12], [32], [35], [2...",angry,V
1,"[[[55], [33], [54], [32], [26], [21], [50], [2...",angry,V
2,"[[[123], [123], [126], [131], [124], [69], [10...",angry,V
3,"[[[255], [255], [255], [255], [255], [255], [2...",angry,V
4,"[[[127], [121], [124], [137], [123], [118], [1...",angry,V


In [6]:
train_data['labels'].value_counts()

happy       7215
neutral     4965
sad         4830
fear        4097
angry       3995
surprise    3171
disgust      436
Name: labels, dtype: int64

In [7]:
happy_count = (train_data['labels'] == 'happy').sum()
print("Happy class count:", happy_count)

if happy_count > 0:
  happy_df = train_data[train_data['labels'] == 'happy'].sample(n=min(happy_count, 3171))
else:
  happy_df = pd.DataFrame()
neutral_count = (train_data['labels'] == 'neutral').sum()
print("neutral class count:", neutral_count)

if neutral_count > 0:
  neutral_df = train_data[train_data['labels'] == 'neutral'].sample(n=min(neutral_count, 3171))
else:
  neutral_df = pd.DataFrame()
sad_count = (train_data['labels'] == 'sad').sum()
print("sad class count:", sad_count)

if sad_count > 0:
  sad_df = train_data[train_data['labels'] == 'sad'].sample(n=min(sad_count, 3171))
else:
  sad_df = pd.DataFrame()
fear_count = (train_data['labels'] == 'fear').sum()
print("fear class count:", fear_count)

if fear_count > 0:
  fear_df = train_data[train_data['labels'] == 'fear'].sample(n=min(fear_count, 3171))
else:
  fear_df = pd.DataFrame()
angry_count = (train_data['labels'] == 'angry').sum()
print("angry class count:", angry_count)

if angry_count > 0:
  angry_df = train_data[train_data['labels'] == 'angry'].sample(n=min(angry_count, 3171))
else:
  angry_df = pd.DataFrame()
surprise_count = (train_data['labels'] == 'surprise').sum()
print("surprise class count:", surprise_count)

if surprise_count > 0:
  surprise_df = train_data[train_data['labels'] == 'surprise'].sample(n=min(surprise_count, 3171))
else:
  surprise_df = pd.DataFrame()
disgust_count = (train_data['labels'] == 'disgust').sum()
print("disgust class count:", disgust_count)

if disgust_count > 0:
  disgust_df = train_data[train_data['labels'] == 'disgust'].sample(n=min(disgust_count, 3171))
else:
  disgust_df = pd.DataFrame()

train_data = pd.concat([happy_df,neutral_df,sad_df,fear_df,angry_df,surprise_df,disgust_df])

train_data = train_data.sample(frac=1)
train_data.reset_index(inplace=True)
train_data.drop('index',inplace=True,axis=1)

train_data.head()

Happy class count: 7215
neutral class count: 4965
sad class count: 4830
fear class count: 4097
angry class count: 3995
surprise class count: 3171
disgust class count: 436


,images,labels,purpose
0,"[[[44], [41], [64], [108], [95], [77], [83], [...",happy,T
1,"[[[113], [114], [108], [110], [90], [65], [84]...",neutral,T
2,"[[[198], [202], [201], [195], [194], [196], [1...",surprise,T
3,"[[[233], [234], [237], [235], [233], [230], [2...",fear,T
4,"[[[79], [74], [91], [82], [92], [94], [74], [7...",angry,T


In [8]:
train_data['labels'].value_counts()

happy       3171
neutral     3171
surprise    3171
fear        3171
angry       3171
sad         3171
disgust      436
Name: labels, dtype: int64

In [9]:
batch_size= 32
classes = 7
rows,columns=48,48

In [10]:
from keras.utils import to_categorical


train_labels = [emotions2int[label] for label in train_data['labels']]
train_labels = to_categorical(train_labels)

val_labels = [emotions2int[label] for label in val_data['labels']]
val_labels = to_categorical(val_labels)


train_data = list(train_data['images'])
train_data = np.array(train_data)

val_data = list(val_data['images'])
val_data = np.array(val_data)

In [11]:
train_data.shape

(19462, 48, 48, 1)

In [12]:
val_data.shape

(7178, 48, 48, 1)

In [13]:
model = Sequential()


model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(256,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(128,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(classes,activation='softmax',kernel_initializer='he_normal'))

print(model.summary())

D:\Users\npuni\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,915,207 (22.56 MB)

 Trainable params: 5,910,471 (22.55 MB)

 Non-trainable params: 4,736 (18.50 KB)

None


In [14]:
checkpoint = ModelCheckpoint('model\\6_class_emotion_detector_V2.keras',
                             save_best_only=True,
                             mode='min',
                             monitor='val_loss',
                             verbose=1)

earlystopping = EarlyStopping(patience=10,
                             verbose=1,
                             min_delta=0,
                             monitor='val_loss',
                             restore_best_weights=True)


callbacks = [checkpoint, earlystopping]

model.compile(metrics=['accuracy'],
             optimizer='rmsprop',
             loss='categorical_crossentropy')

In [15]:
train_samples = 28273
validation_samples = 3534
batch_size = 64
epochs=11

history = model.fit(train_data,
                    train_labels,
                    epochs=epochs,
                    steps_per_epoch=train_samples//batch_size,
                    validation_data=(val_data,val_labels),
                    validation_steps=validation_samples//batch_size,
                    callbacks=callbacks)

Epoch 1/11
433/441 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.1748 - loss: 2.5819 

D:\Users\npuni\anaconda3\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 1.85670, saving model to model\6_class_emotion_detector_V2.keras
441/441 ━━━━━━━━━━━━━━━━━━━━ 587s 1s/step - accuracy: 0.1749 - loss: 2.5769 - val_accuracy: 0.2403 - val_loss: 1.8567
Epoch 2/11
433/441 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.2265 - loss: 1.8917
Epoch 2: val_loss improved from 1.85670 to 0.00000, saving model to model\6_class_emotion_detector_V2.keras
441/441 ━━━━━━━━━━━━━━━━━━━━ 435s 986ms/step - accuracy: 0.2270 - loss: 1.8903 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/11
433/441 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.3587 - loss: 1.6285
Epoch 3: val_loss did not improve from 0.00000
441/441 ━━━━━━━━━━━━━━━━━━━━ 509s 1s/step - accuracy: 0.3591 - loss: 1.6275 - val_accuracy: 0.4324 - val_loss: 1.4371
Epoch 4/11
433/441 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.4431 - loss: 1.4469 
Epoch 4: val_loss did not improve from 0.00000
441/441 ━━━━━━━━━━━━━━━━━━━━ 529s 1s/step - accuracy: 0.4431 - loss: 1.4

In [16]:
import cv2
from tensorflow.keras.models import load_model 
import numpy as np

int2emotions = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise', 6: 'Disgust'}
emotion_text_colors = {'Angry': (0, 0, 255), 'Fear': (0, 128, 255), 'Happy': (0, 255, 0),
                       'Neutral': (255, 255, 0), 'Sad': (255, 0, 0), 'Surprise': (255, 165, 0),
                       'Disgust': (128, 0, 128)}
emotion_bg_colors = {'Angry': (255, 255, 255), 'Fear': (255, 255, 255), 'Happy': (255, 255, 255),
                     'Neutral': (255, 255, 255), 'Sad': (255, 255, 255), 'Surprise': (255, 255, 255),
                     'Disgust': (255, 255, 255)}

model = load_model('model/6_class_emotion_detector_V2.keras') 

classifier = cv2.CascadeClassifier(r"C:\Users\karth\OneDrive\Desktop\emotion 3\haarcascade_frontalface_default.xml")

def add_transparent_text_bg(img, text, pos, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, font_thickness=2, text_color=(0, 0, 0), bg_color=(255, 255, 255), bg_alpha=0.5):
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_x, text_y = pos
    overlay = img[text_y:text_y + text_size[1] + 2, text_x:text_x + text_size[0] + 2]
    cv2.putText(img, text, pos, font, font_scale, text_color, font_thickness)

def detect_face(frame):
    if classifier.empty():
        print("Error: Could not load face cascade classifier.")
        return frame

    faces = classifier.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2) 

        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)  
        face_roi = cv2.resize(face_roi, (48, 48)) 
        face_roi = face_roi.astype('float32') / 255.0 
        face_roi = np.expand_dims(face_roi, axis=0) 

        predicted_emotion = int2emotions[np.argmax(model.predict(face_roi))]
        text_color = emotion_text_colors[predicted_emotion]
        bg_color = emotion_bg_colors[predicted_emotion]

        add_transparent_text_bg(frame, predicted_emotion, (x, y - 30), text_color=text_color, bg_color=bg_color)

    return frame

cap = cv2.VideoCapture(0)

cv2.namedWindow('Emotion Detector', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Emotion Detector', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    ret, frame = cap.read()
    if ret:
        frame = detect_face(frame)
        cv2.imshow('Emotion Detector', frame)
        if cv2.waitKey(1) == 27:
            break

cap.release()
cv2.destroyAllWindows()




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━